# Monte Carlo simulation on 3D Ising model

This is a 3D Ising model, extending from the 2D case (Ref. 2D Ising model) 


In [2]:
#----------------------------------------------------------------------#
#  Import modules
#----------------------------------------------------------------------#
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.animation as animation


#----------------------------------------------------------------------#
#   Define a function myFloat transform myList to float
#----------------------------------------------------------------------#
def myFloat(myList):
        return map(float, myList)


#----------------------------------------------------------------------#
#   Monte Carlo simulation parameters
#----------------------------------------------------------------------#
SIZE = 5                          # MC size SIZE * SIZE
STEPS = 10000                     # MC steps
TEMP = 0.1                         # MC temperature (unit: K)
J = 1.0                            # Heisenberg interaction parameter (unit: meV)
V = 1

#----------------------------------------------------------------------#
#  Check periodic boundary conditions 
#----------------------------------------------------------------------#
def bc(i):
    if i+1 > SIZE-1:
        return 0
    if i-1 < 0:
        return SIZE-1
    else:
        return i


#----------------------------------------------------------------------#
#  Build system
#----------------------------------------------------------------------#
def build_system():
    #np.random.seed(1)
    system = np.random.randint(2,size=(SIZE,SIZE,SIZE))
    system[system==0] = -1

    return system

init_system = build_system()        # Initial spin order


#----------------------------------------------------------------------#
#  Calculate energy (nearest interaction)
#----------------------------------------------------------------------#
def energy(system, X, Y, Z):
    eng = -1 * system[X, Y, Z] * (
            J * (system[bc(X+1), Y, Z] + system[bc(X-1), Y, Z] +
                    system[X, bc(Y+1), Z] + system[X, bc(Y-1), Z] +
                    system[X, Y, bc(Z+1)] + system[X, Y, bc(Z-1)]))
    return eng


#----------------------------------------------------------------------#
#   Monte carlo loops
#----------------------------------------------------------------------#
system = build_system()

f = open('mc_data_3d', 'w')            # Spin order of each accepted MC step
n_acpt = 0                          # Number of accepted MC steps

for step, x in enumerate(range(STEPS)):
    X = np.random.randint(0,SIZE)
    Y = np.random.randint(0,SIZE)
    Z = np.random.randint(0,SIZE)

    E = -2. * energy(system, X, Y, Z)

    if E < 0.:
        system[X, Y, Z] *= -1
        for i in system:
            np.savetxt(f,i,fmt='%3d')
        n_acpt += 1

    elif np.exp(-1./TEMP*E) > np.random.rand():
        system[X, Y, Z] *= -1
        for i in system:
            np.savetxt(f,i,fmt='%3d')
        n_acpt += 1

f.close()


#----------------------------------------------------------------------#
#   Monte carlo outputs
#----------------------------------------------------------------------#
def outputs():
    print '='*70
    print 'Number of accepted steps: {:6d}'.format(n_acpt)
    print 'Spin order of every MC step is save in file mc_data_3d'
    print '='*70,'\n'

    print '='*70
    print 'The initial spin order'
    print '='*70,'\n'
    print init_system,'\n'

    print '='*70
    print 'The final spinal order'
    print '='*70,'\n'
    print system,'\n'

outputs()

Number of accepted steps:    153
Spin order of every MC step is save in file mc_data_3d

The initial spin order

[[[-1 -1 -1  1  1]
  [-1 -1  1  1  1]
  [-1  1 -1  1 -1]
  [-1  1  1 -1 -1]
  [-1  1 -1 -1  1]]

 [[-1 -1  1  1  1]
  [ 1 -1  1  1  1]
  [ 1  1  1 -1 -1]
  [ 1  1  1  1 -1]
  [-1 -1 -1  1  1]]

 [[-1  1  1 -1  1]
  [-1  1 -1 -1 -1]
  [-1 -1 -1  1 -1]
  [-1 -1 -1  1 -1]
  [-1 -1  1  1 -1]]

 [[-1 -1  1 -1  1]
  [ 1 -1  1  1  1]
  [-1  1 -1  1 -1]
  [ 1 -1 -1  1 -1]
  [-1  1  1  1  1]]

 [[ 1  1  1  1  1]
  [ 1 -1 -1  1 -1]
  [-1  1 -1  1  1]
  [ 1  1  1 -1 -1]
  [ 1 -1 -1 -1 -1]]] 

The final spinal order

[[[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]]] 



## Visualize spin configurations with 3d scattering points

In [4]:
#----------------------------------------------------------------------#
#   Plot 3d spin order
#----------------------------------------------------------------------#
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.view_init(azim=80)

x, y, z = np.meshgrid(np.arange(0, SIZE, 1),
                        np.arange(0, SIZE, 1),
                        np.arange(0, SIZE, 1))

C = init_system[x, y, z]
colors = C.reshape(1,-1)[0]

ax.set_xlim(0, SIZE-1)
ax.set_ylim(0, SIZE-1)
ax.set_zlim(0, SIZE-1)

S = ax.scatter(x, y, z,
        cmap = plt.cm.seismic,
        s = 100,
        alpha = 0.6,
        edgecolors = 'none')
S.set_array(colors)

f = open("mc_data_3d","r")
out = [line.split() for line in f]
data = map(myFloat,out)
f.close()

def animate(i):
    C = np.asarray(data[i*SIZE*SIZE:(i+1)*SIZE*SIZE])
    c = C.reshape(1,-1)[0]
    S.set_array(c)
    return S

ani = animation.FuncAnimation(fig, animate, interval=V, save_count=n_acpt)
#ani.save('3d_points.gif', writer="imagemagick", fps=10, dpi=100)
#ani.save('3d_points.mp4', writer="ffmpeg", fps=10, dpi=300)
#plt.show()

<video width="640" height="480" controls src="3d_points.mp4" />